In [1]:
!pip install tensorboardx
!pip install transformers
!pip install simpletransformers

     |████████████████████████████████| 204kB 5.0MB/s 
     |████████████████████████████████| 645kB 4.8MB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
     |████████████████████████████████| 1.1MB 54.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a69ee072b3c4876708f063076efa5e7745a7ef11557b1a408d3f8100b1b58e17
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 4.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=968e3b6f73e56795f96ac6e1f6e573ea185e92dc3cfb7b003ba51b46820f1937
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [2]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir  ./apex

Cloning into 'apex'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 6734 (delta 0), reused 1 (delta 0), pack-reused 6726
Receiving objects: 100% (6734/6734), 13.74 MiB | 28.48 MiB/s, done.
Resolving deltas: 100% (4494/4494), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-4xptsqbo
Created temporary directory: /tmp/pip-req-tracker-2e8wriga
Created requirements tracker '/tmp/pip-req-tracker-2e8wriga'
Created temporary directory: /tmp/pip-install-fwgzjssi
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-81jv9bm7
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-2e8wriga'
    Running setup.py (path:/tmp/pip-req-build-81jv9bm7/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-81jv9bm7/pip-egg-info/apex.e

In [3]:
import pandas as pd
import numpy as np
from transformers import *

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
stop = stopwords.words('english')
stop.pop(118)

from sklearn.model_selection import train_test_split

from simpletransformers.classification import ClassificationModel

from sklearn.metrics import mean_absolute_error, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
read_path = "/content/drive/My Drive/cs182final/yelp_review_training_dataset.jsonl"
data = pd.read_json(read_path, lines = True)

In [0]:
data = data.drop(["review_id"], axis =1)

In [7]:
data.head()

,text,stars
0,Total bill for this horrible service? Over $8G...,1
1,I *adore* Travis at the Hard Rock's new Kelly ...,5
2,I have to say that this office really has it t...,5
3,Went in for a lunch. Steak sandwich was delici...,5
4,Today was my second out of three sessions I ha...,1


In [8]:
data["text"] = data['text'].str.replace('[^\w\s]','').str.lower()
data["text"] = data['text'].str.replace('\d+', '')
data['text'] = data['text'].apply(lambda x: [item for item in x.split() if item not in stop])
data['text'] = data['text'].apply(lambda x: ' '.join(x))

data.head()

,text,stars
0,total bill horrible service gs crooks actually...,1
1,adore travis hard rocks new kelly cardenas sal...,5
2,say office really together organized friendly ...,5
3,went lunch steak sandwich delicious caesar sal...,5
4,today second three sessions paid although firs...,1


In [9]:
data.sample(frac=1)
data_sample = data.iloc[:10000]
data_sample.columns = ['text', 'labels']
train, validation = train_test_split(data_sample, test_size=0.2)
data_sample.shape

(10000, 2)

In [10]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=6, use_cuda=True, args = {'output_dir' : "outputs_roberta/", "overwrite_output_dir" : True})

In [11]:
model.train_model(train)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.759040

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.650070

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.156027Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.171499Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.766126Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.620604Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Running loss: 1.031548


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [12]:
result, model_outputs, wrong_predictions = model.eval_model(validation, MAE=mean_absolute_error, acc=accuracy_score)

In [13]:
print(result)

{'mcc': 0.5723544056597207, 'MAE': 0.4085, 'acc': 0.7015, 'eval_loss': 0.7503936570882798}


In [14]:
print(model_outputs)

[[-2.8378906  -2.4199219  -1.8896484   0.37548828  2.8476562   3.8984375 ]
 [-3.7304688   0.07617188 -0.3828125   0.27001953  1.2265625   2.4355469 ]
 [-2.40625    -1.5048828  -2.0019531  -0.8232422   1.8017578   5.125     ]
 ...
 [-2.8203125  -1.9375     -1.9580078  -0.16699219  2.4335938   4.46875   ]
 [-2.4707031  -1.4541016  -1.9355469  -0.7841797   1.7998047   5.0859375 ]
 [-2.703125   -1.2734375  -1.8330078  -0.77783203  1.7060547   4.9882812 ]]


In [15]:
print(wrong_predictions)

[<simpletransformers.classification.classification_utils.InputExample object at 0x7f39a47d6898>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9e80>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c92b0>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c90f0>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9320>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9550>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c95f8>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9860>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9898>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f39a67c9940>, <simpletransformers.classification.clas

In [16]:
model_outputs.shape

(2000, 6)

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [18]:
train.head()

,text,labels
4226,not sure good got coupon try went closed cater...,1
8636,wish could leave zero stars always get order w...,1
2434,wish could give negative stars restaurant far ...,1
4094,took children visiting sister champaign expect...,2
8208,horrendously expensive chocolate paid two tiny...,2


In [19]:
train = pd.get_dummies(train, columns = ['labels'])
train.head()

,text,labels_1,labels_2,labels_3,labels_4,labels_5
4226,not sure good got coupon try went closed cater...,1,0,0,0,0
8636,wish could leave zero stars always get order w...,1,0,0,0,0
2434,wish could give negative stars restaurant far ...,1,0,0,0,0
4094,took children visiting sister champaign expect...,0,1,0,0,0
8208,horrendously expensive chocolate paid two tiny...,0,1,0,0,0


In [20]:
validation = pd.get_dummies(validation, columns = ['labels'])
train.shape, validation.shape

((8000, 6), (2000, 6))

In [21]:
class_names = ['labels_1', 'labels_2', 'labels_3', 'labels_4', 'labels_5']
# Splitting off y variable
y = train[class_names].values
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [0]:
# max number of unique words 
max_features = 20000
# max number of words from review to use
maxlen = 200
# batch size
batchsize = 32

In [23]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train['text'].values))
X_train = tokenizer.texts_to_sequences(train['text'].values)
X_validation = tokenizer.texts_to_sequences(validation['text'].values)
x_train = pad_sequences(X_train, maxlen = maxlen)
x_validation = pad_sequences(X_validation, maxlen = maxlen)
x_train

array([[   0,    0,    0, ..., 1596,  558,  444],
       [   0,    0,    0, ...,   18,  100,   12],
       [   0,    0,    0, ..., 1788,   76,   45],
       ...,
       [   0,    0,    0, ..., 2713, 2413,  352],
       [   0,    0,    0, ...,  966,  446,  153],
       [   0,    0,    0, ...,  167,  873,  414]], dtype=int32)

In [24]:
x_train.shape

(8000, 200)

In [0]:
model_LSTM = Sequential()

In [0]:
model_LSTM.add(Embedding(*model_outputs.shape, weights=[model_outputs], input_length=200, trainable=False))

In [0]:
model_LSTM.add(Bidirectional(LSTM(50)))
model_LSTM.add(Dropout(0.1))
model_LSTM.add(Dense(5, activation='softmax'))
model_LSTM.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model_LSTM.fit(x_train, y, batch_size = batchsize, epochs = 10)

Epoch 1/10
8000/8000 [==============================] - 151s 19ms/step - loss: 1.4077 - accuracy: 0.4546
Epoch 2/10
8000/8000 [==============================] - 149s 19ms/step - loss: 1.3901 - accuracy: 0.4608
Epoch 3/10
8000/8000 [==============================] - 148s 18ms/step - loss: 1.3795 - accuracy: 0.4627
Epoch 4/10
8000/8000 [==============================] - 152s 19ms/step - loss: 1.3766 - accuracy: 0.4650
Epoch 5/10
8000/8000 [==============================] - 152s 19ms/step - loss: 1.3698 - accuracy: 0.4684
Epoch 6/10
8000/8000 [==============================] - 149s 19ms/step - loss: 1.3623 - accuracy: 0.4711
Epoch 7/10
8000/8000 [==============================] - 148s 18ms/step - loss: 1.3546 - accuracy: 0.4706
Epoch 8/10
8000/8000 [==============================] - 145s 18ms/step - loss: 1.3480 - accuracy: 0.4749
Epoch 9/10
8000/8000 [==============================] - 143s 18ms/step - loss: 1.3410 - accuracy: 0.4787
Epoch 10/10
8000/8000 [==============================] 

In [29]:
y_validation = model_LSTM.predict([x_validation], batch_size=batchsize, verbose = 1)

2000/2000 [==============================] - 4s 2ms/step


In [30]:
model_LSTM.evaluate(x_validation, validation[class_names].values, verbose = 1, batch_size=batchsize)

2000/2000 [==============================] - 4s 2ms/step


[1.3509935522079468, 0.4740000069141388]